## Advent of Code - Day 23

## Part 1

In [1]:
import collections


In [2]:
test_str='389125467'

In [3]:
deque_in=collections.deque([int(x) for x in test_str])
deque_in

deque([3, 8, 9, 1, 2, 5, 4, 6, 7])

In [4]:
def pick_3(state_in):
    state_in.rotate(-1)
    picked_up=(state_in.popleft(), state_in.popleft(), state_in.popleft())
    state_in.rotate()
    
    return picked_up

In [5]:
def find_destination(state_in):
    current=state_in.popleft()
    if min(state_in)>current:
        destination=max(state_in)
    else:
        destination=max([x for x in state_in
                         if x<current])
    state_in.appendleft(current)

    return destination

In [6]:
def insert_at_destination(destination, state_in, picked_up):
    if state_in[0]==destination:
        state_in.rotate(-1)
        (x, y, z)=picked_up
        state_in.appendleft(z)
        state_in.appendleft(y)
        state_in.appendleft(x)
        state_in.rotate()
    else:
        state_in.rotate(-1)
        insert_at_destination(destination, state_in, picked_up)
        state_in.rotate()

In [7]:
def move(state_in):
    
    picked_up=pick_3(state_in)
    insert_at_destination(find_destination(state_in),
                          state_in,
                          picked_up)
    state_in.rotate(-1)

In [8]:
def day23_part1(str_in, n=100):
    
    state=collections.deque([int(x) for x in str_in])

    for i in range(n):
        move(state)
        
    while state[0]!=1:
        state.rotate()
        
    return ''.join(str(x) for x in state)[1:]
    

In [9]:
assert day23_part1('389125467', 10)=='92658374'
assert day23_part1('389125467', 100)=='67384529'

In [10]:
day23_part1('925176834', n=100)

'69852437'

## Part 2

This is taking a bit of time, partly because there are bits of python where I'm not entirely sure what's going on under the bonnet. This is needing a pretty significant rewrite of part 1.

I assume that there's some clever way of seeing a pattern here, but it's escaping me at the moment.

So it has come to this. Using objects. I feel sullied.

In [11]:
class Cups():
    
    def __init__(self, str_in, n=0):
        
        if n<len(str_in):
            self.length=len(str_in)
        else:
            self.length=n
            
        self.values=[int(x) for x in str_in]
        self.values.extend(range(len(str_in)+1, self.length+1))
        
        self.next_cup=list(range(1, self.length))
        self.next_cup.append(0)
                
        self.current=0
        
    def find_value_idx(self, value):
        if value<10:
            return self.values.index(value)
        else:
            return value-1
        
    def find_destination(self, picked_values):
        c=self.values[self.current]
        
        while True:
            c=c-1
            if not c:
                c=self.length
            if c not in picked_values:
                return c
        
    
    def move(self):
        
        # Find the three values to move
        c1_idx=self.next_cup[self.current]
        c2_idx=self.next_cup[c1_idx]
        c3_idx=self.next_cup[c2_idx]
        picked_values=(self.values[c1_idx],
                       self.values[c2_idx],
                       self.values[c3_idx])
        
        
        # Find where to put them
        destination_value=self.find_destination(picked_values)
        destination_idx=self.find_value_idx(destination_value)

        # Point the current cup to where cup 3 was pointing
        self.next_cup[self.current]=self.next_cup[c3_idx]

        # Point the destination cup at cup 1:
        destination_next_cup=self.next_cup[destination_idx]
        self.next_cup[destination_idx]=c1_idx
        
        # and point c3 to where the destination was pointing
        self.next_cup[c3_idx]=destination_next_cup
        
        # Finally, move to the next cup:
        self.current=self.next_cup[self.current]
        
        return destination_value
    
    def show(self):
        
        print('(' + str(self.values[self.current]) + ') ', end='')
        i=self.next_cup[self.current]

        while i!=self.current:
            print(self.values[i], end=' ')
            i=self.next_cup[i]
        print()
        
    def advance_to(self, destination_value):
        self.current=self.find_value_idx(destination_value)
            
    def next_n_values(self, n):
        d=self.current
        out_ls=[]
        for i in range(n):
            d=self.next_cup[d]
            out_ls.append(self.values[d])
        return out_ls
            

In [12]:
c=Cups('925176834', 1000000)

for i in range(10000000):
    c.move()
    
c.advance_to(1)
c.next_n_values(5)

[351591, 259985, 883531, 597030, 933448]

In [13]:
[x, y]=c.next_n_values(2)
x*y

91408386135

Done.